In [202]:
!wget -nc https://raw.githubusercontent.com/lazyprogrammer/machine_learning_examples/master/hmm_class/edgar_allan_poe.txt
!wget -nc https://raw.githubusercontent.com/lazyprogrammer/machine_learning_examples/master/hmm_class/robert_frost.txt

File ‘edgar_allan_poe.txt’ already there; not retrieving.

File ‘robert_frost.txt’ already there; not retrieving.



In [203]:
import numpy as np
import matplotlib.pyplot as plt
import string
from sklearn.model_selection import train_test_split

In [204]:
input_files = [
  'edgar_allan_poe.txt',
  'robert_frost.txt',
]

In [205]:
!head edgar_allan_poe.txt

LO! Death hath rear'd himself a throne
In a strange city, all alone,
Far down within the dim west
Where the good, and the bad, and the worst, and the best,
Have gone to their eternal rest.
 
There shrines, and palaces, and towers
Are not like any thing of ours
Oh no! O no! ours never loom
To heaven with that ungodly gloom!


In [206]:
!head robert_frost.txt

Two roads diverged in a yellow wood,
And sorry I could not travel both
And be one traveler, long I stood
And looked down one as far as I could
To where it bent in the undergrowth; 

Then took the other, as just as fair,
And having perhaps the better claim
Because it was grassy and wanted wear,
Though as for that the passing there


In [207]:
# collect data into lists
input_texts = []
labels = []

for label, f in enumerate(input_files):
  print(f"{f} corresponds to label {label}")

  for line in open(f):
    line = line.rstrip().lower()
    if line:
      # remove punctuation
      line = line.translate(str.maketrans('', '', string.punctuation))

      input_texts.append(line)
      labels.append(label)

edgar_allan_poe.txt corresponds to label 0
robert_frost.txt corresponds to label 1


In [208]:
train_text, test_text, Ytrain, Ytest = train_test_split(input_texts, labels)

In [209]:
len(Ytrain), len(Ytest)

(1615, 539)

In [210]:
train_text[:5]

['like starlight on a pall',
 'and in a town that is no more a town',
 'who otherwise would fall from life and heaven',
 'the frogs that were peeping a thousand shrill',
 'then cane one knock']

In [211]:
Ytrain[:5]

[0, 1, 0, 1, 1]

In [212]:
idx = 1
word2idx = {'<unk>': 0}

In [213]:
# populate word2idx
for text in train_text:
    tokens = text.split()
    for token in tokens:
      if token not in word2idx:
        word2idx[token] = idx
        idx += 1

In [214]:
word2idx

{'<unk>': 0,
 'like': 1,
 'starlight': 2,
 'on': 3,
 'a': 4,
 'pall': 5,
 'and': 6,
 'in': 7,
 'town': 8,
 'that': 9,
 'is': 10,
 'no': 11,
 'more': 12,
 'who': 13,
 'otherwise': 14,
 'would': 15,
 'fall': 16,
 'from': 17,
 'life': 18,
 'heaven': 19,
 'the': 20,
 'frogs': 21,
 'were': 22,
 'peeping': 23,
 'thousand': 24,
 'shrill': 25,
 'then': 26,
 'cane': 27,
 'one': 28,
 'knock': 29,
 'two': 30,
 'oldbelievers': 31,
 'they': 32,
 'did': 33,
 'all': 34,
 'talking': 35,
 'nor': 36,
 'never': 37,
 'will': 38,
 'be': 39,
 'to': 40,
 'my': 41,
 'way': 42,
 'of': 43,
 'thinking': 44,
 'just': 45,
 'oer': 46,
 'bright': 47,
 'island': 48,
 'smile': 49,
 'but': 50,
 'why': 51,
 'when': 52,
 'shes': 53,
 'well': 54,
 'off': 55,
 'it': 56,
 'neighbours': 57,
 'hours': 58,
 'flew': 59,
 'brightly': 60,
 'by': 61,
 'perhaps': 62,
 'you': 63,
 'have': 64,
 'art': 65,
 'what': 66,
 'i': 67,
 'mean': 68,
 'was': 69,
 'some': 70,
 'money': 71,
 'suddenly': 72,
 'come': 73,
 'into': 74,
 'watch': 75

In [215]:
len(word2idx)

2523

In [216]:
# convert data into integer format
train_text_int = []
test_text_int = []

for text in train_text:
  tokens = text.split()
  line_as_int = [word2idx[token] for token in tokens]
  train_text_int.append(line_as_int)

for text in test_text:
  tokens = text.split()
  line_as_int = [word2idx.get(token, 0) for token in tokens]
  test_text_int.append(line_as_int)

In [217]:
train_text_int[100:105]

[[6, 387, 160, 55, 20, 244, 117, 388],
 [67, 389, 170, 390, 390, 169, 391, 392],
 [67, 159, 20, 263, 43, 4, 393],
 [394, 206, 395, 40, 144, 171, 238, 4, 140],
 [26, 232, 396, 289, 171, 80, 28, 397, 398]]

In [218]:
# initialize A and pi matrices - for both classes
V = len(word2idx)

A0 = np.ones((V, V))
pi0 = np.ones(V)

A1 = np.ones((V, V))
pi1 = np.ones(V)

In [219]:
# compute counts for A and pi
def compute_counts(text_as_int, A, pi):
  for tokens in text_as_int:
    last_idx = None
    for idx in tokens:
      if last_idx is None:
        # it's the first word in a sentence
        pi[idx] += 1
      else:
        # the last word exists, so count a transition
        A[last_idx, idx] += 1

      # update last idx
      last_idx = idx


compute_counts([t for t, y in zip(train_text_int, Ytrain) if y == 0], A0, pi0)
compute_counts([t for t, y in zip(train_text_int, Ytrain) if y == 1], A1, pi1)

In [220]:
# normalize A and pi so they are valid probability matrices
# convince yourself that this is equivalent to the formulas shown before
A0 /= A0.sum(axis=1, keepdims=True)
pi0 /= pi0.sum()

A1 /= A1.sum(axis=1, keepdims=True)
pi1 /= pi1.sum()

In [221]:
# log A and pi since we don't need the actual probs
logA0 = np.log(A0)
logpi0 = np.log(pi0)

logA1 = np.log(A1)
logpi1 = np.log(pi1)

In [222]:
# compute priors
count0 = sum(y == 0 for y in Ytrain)
count1 = sum(y == 1 for y in Ytrain)
total = len(Ytrain)
p0 = count0 / total
p1 = count1 / total
logp0 = np.log(p0)
logp1 = np.log(p1)
p0, p1

(0.33126934984520123, 0.6687306501547987)

In [223]:
# build a classifier
class Classifier:
  def __init__(self, logAs, logpis, logpriors):
    self.logAs = logAs
    self.logpis = logpis
    self.logpriors = logpriors
    self.K = len(logpriors) # number of classes

  def _compute_log_likelihood(self, input_, class_):
    logA = self.logAs[class_]
    logpi = self.logpis[class_]

    last_idx = None
    logprob = 0
    for idx in input_:
      if last_idx is None:
        # it's the first token
        logprob += logpi[idx]
      else:
        logprob += logA[last_idx, idx]

      # update last_idx
      last_idx = idx

    return logprob

  def predict(self, inputs):
    predictions = np.zeros(len(inputs))
    for i, input_ in enumerate(inputs):
      posteriors = [self._compute_log_likelihood(input_, c) + self.logpriors[c] \
             for c in range(self.K)]
      pred = np.argmax(posteriors)
      predictions[i] = pred
    return predictions

In [224]:
# each array must be in order since classes are assumed to index these lists
clf = Classifier([logA0, logA1], [logpi0, logpi1], [logp0, logp1])

In [225]:
Ptrain = clf.predict(train_text_int)
print(f"Train acc: {np.mean(Ptrain == Ytrain)}")

Train acc: 0.9950464396284829


In [226]:
Ptest = clf.predict(test_text_int)
print(f"Test acc: {np.mean(Ptest == Ytest)}")

Test acc: 0.8367346938775511


In [227]:
from sklearn.metrics import confusion_matrix, f1_score

# read about F-score: https://en.wikipedia.org/wiki/F-score

In [228]:
cm = confusion_matrix(Ytrain, Ptrain)
cm

array([[ 527,    8],
       [   0, 1080]])

In [229]:
cm_test = confusion_matrix(Ytest, Ptest)
cm_test

array([[102,  81],
       [  7, 349]])

In [230]:
'f1 score for pos_label = 1'
f1_score(Ytrain, Ptrain)

0.996309963099631

In [231]:
'f1 score simple average for pos_label = 0 and pos_label = 1'
f1_score(Ytrain, Ptrain,average = 'macro')

0.9943885032070661

In [232]:
'f1 score sample size weighted average for pos_label = 0 and pos_label = 1'
f1_score(Ytrain, Ptrain,average = 'weighted')

0.9950369215609036

In [233]:
'f1 score for pos_label = 1'
f1_score(Ytest, Ptest)

0.8880407124681933

In [234]:
'f1 score simple average for pos_label = 0 and pos_label = 1'
f1_score(Ytest, Ptest,average = 'macro')

0.7933354247272473

In [235]:
'f1 score sample size weighted average for pos_label = 0 and pos_label = 1'
f1_score(Ytest, Ptest,average = 'weighted')

0.8237324836867718

In [236]:
'class wise label distribution in Ytest'
count0 = sum(y == 0 for y in Ytest)
count1 = sum(y == 1 for y in Ytest)
n = len(Ytest)
count0/n,count1/n

(0.3395176252319109, 0.660482374768089)

In [237]:
'class wise label distribution in Ytrain'
count0 = sum(y == 0 for y in Ytrain)
count1 = sum(y == 1 for y in Ytrain)
n = len(Ytrain)
count0/n,count1/n

(0.33126934984520123, 0.6687306501547987)